In [ ]:
import numpy as np
import math
from scipy.linalg import dft 

In [1]:
# # test grad and stoch grad speed
# import numpy as np
# import matplotlib.pyplot as plt
# from os.path import dirname, abspath
# import os
# import matplotlib.pyplot as plt
# from PIL import Image
# import time

# from skimage import data, img_as_float
# from skimage.restoration import denoise_nl_means, estimate_sigma
# from skimage.metrics import peak_signal_noise_ratio
# from skimage.util import random_noise


# X = np.random.rand(256,256)
# Y = np.random.rand(256,256)
# mmask = np.ones((256,256))
# ind = np.zeros((256,256))
# ind[50,50] = 1
# ind[10,20] = 1
# ind[10,30] = 1
# ind[10,40] = 1

# start_full = time.time()
# full_grad(X, mmask, Y)
# stop_full = time.time()

# start_stoch = time.time()
# stoch_grad(X, ind, Y)
# stop_stoch = time.time()

# print('full', stop_full-start_full)
# print('stoch', stop_stoch-start_stoch)

In [2]:
def full_grad(z, MASK, meas):
    # Input: 
    # z, optimization iterate
    # MASK, observed fourier measurements
    # meas, measurements = F(X) + w
    # Output:
    # Full gradient at z

    # real grad
    H, W = z.shape[:2]
    res = np.fft.fft2(z) * MASK
    index = np.nonzero(MASK)
    res[index] = res[index] - meas[index]
    return np.real(np.fft.ifft2(res))

In [3]:
def stoch_grad(z, IND, meas):
    # Input:
    # z, optimization iterate
    # meas, measurements = F(X) + w
    # batch_index, indices to update
    # Output:
    # stochastic gradient at z for measurements in B
    H, W = z.shape[:2]
    # batch gradient update
    res = IND * (np.fft.fft2(z) - meas)
    return np.real(np.fft.ifft2(res)) 

In [4]:
def get_batch(B, H, W, nnz_list):
    tmp = np.random.permutation(nnz_list)
    k = tmp[0:B][:]
    batch = np.zeros([H,W])
    batch[k[:,0], k[:,1]] = 1

    # find nonzero batch indices
    return batch

    # Input:
    # B, batch size
    # H, image height
    # W, image width
    # N, list of indices
    # Output:
    # batch indices

    #Generate random indices